# API 모듈화

In [250]:
import requests
import pandas as pd

# 데이터 개수 측정
def DataCount(Decoding_key, pagenum, numofrows):
    url = 'http://apis.data.go.kr/1741000/EmergencyAssemblyArea_Earthquake2/getArea1List'
    params ={'serviceKey' : Decoding_key, 'pageNo' : pagenum, 'numOfRows' : numofrows, 'type' : 'json' }
    response = requests.get(url, params=params)
    jsondata = response.json()
    
    total_count = jsondata['EarthquakeOutdoorsShelter'][0]['head'][0]['totalCount']
    total_count = int(total_count)
    return total_count
    
# 옥외 대피소 API 호출
def API_EarthQuake_json(Decoding_key, pagenum, numofrows):
    # API 호출
    url = 'http://apis.data.go.kr/1741000/EmergencyAssemblyArea_Earthquake2/getArea1List'
    params ={'serviceKey' : Decoding_key, 'pageNo' : pagenum, 'numOfRows' : numofrows, 'type' : 'json' }
    response = requests.get(url, params=params)
    jsondata = response.json()
    # 호출 output 형태
    # 'EarthquakeOutdoorsShelter' : 결과 형태
    # 'row'
    
    # json 
    rows = jsondata['EarthquakeOutdoorsShelter'][1]['row']
    df = pd.DataFrame(rows)

    # Data_PreProcessing
    data = df[['vt_acmdfclty_nm','dtl_adres','fclty_ar','xcord','ycord']]

    # data rename
    data = data.rename(columns = {'vt_acmdfclty_nm' : '시설명', 'dtl_adres' : '상세주소', 'fclty_ar' : '시설면적', 'xcord' : '경도', 'ycord' : '위도'})

    # 상세주소 -> 광역시도, 시군구, 읍면동, 주소번호 split -> 상세주소 drop
    data[['광역시도','시군구', '읍면동', '주소번호']] = data['상세주소'].str.split(' ', expand=True)[[0,1,2,5]]
    data = data.drop(['상세주소'], axis=1)
    data = data[['광역시도','시군구', '읍면동', '주소번호', '경도', '위도', '시설명', '시설면적']]
    
    return data

# 전체 옥외 대피소 API 호출
def API_EarthQuake_Total(decoding, pagenum, numofrows):
    total_count = DataCount(decoding, pagenum, numofrows)

    if total_count % numofrows == 0:
        num = total_count//numofrows
    else:
        num = total_count//numofrows + 1
    
    total_df = pd.DataFrame({})
    
    for i in range(1, num+1):
        if i == num: # 마지막 페이지인 경우
            last_numofrows = total_count - (numofrows*(num-1))
            data = API_EarthQuake_json(decoding, i, last_numofrows)
        else: # 마지막 페이지 아닌경우
            data = API_EarthQuake_json(decoding, i, numofrows)
            
        total_df = pd.concat([data, total_df])

    return total_df
# 전국 인원 API 호출
#def API_personnel(Decoding_key):

# 파라미터 변경사항

In [252]:
# API 서비스 키
decoding = 'tFlRbdLrGdbT07si0ZGVgNB2t7e1lRlQPzeq5uCeXyfkCZJ+8+CRvlRy1JLFEZXUdVqNxgmO1OaOcSDStjOhKQ=='
# pagenum
pagenum = 1
# 불러올 개수 (최대 1001개)
numofrows = 1000

total_df = API_EarthQuake_Total(decoding, pagenum, numofrows)

In [253]:
total_df

,광역시도,시군구,읍면동,주소번호,경도,위도,시설명,시설면적
0,전라남도,진도군,고군면,None,126.3418636916799,34.49183897963677,오산초등학교 운동장,4062
1,서울특별시,성북구,동소문동4가,278-174,127.00852664490648,37.5915134599658,삼선중학교 운동장,4608
2,서울특별시,성북구,동소문동6가,27-1,127.01335020874373,37.593152617297534,돈암초등학교 운동장,3406
3,서울특별시,성북구,동소문동7가,118-1,127.01395694361783,37.59806215220391,정덕초등학교 운동장,2417
4,서울특별시,성북구,삼선동3가,43-1,127.0149955498448,37.585698514669694,경동고등학교 운동장,10750
...,...,...,...,...,...,...,...,...
995,대구광역시,수성구,만촌동,1425,128.64046580369416,35.870091421653704,중앙초등학교,2779
996,대구광역시,수성구,만촌동,82-4,128.651033,35.8773108,동문고등학교,12255
997,대구광역시,수성구,만촌동,852,128.64888939019204,35.856495876132875,국화공원,1408
998,대구광역시,수성구,만촌동,991-8,128.647743,35.8654520,무열공원,659


In [116]:
# data frame 생성 후 계속 append -> duplicated로 중복 제거 하는 코드


# SQL 연결

# API 호출 형태 확인

In [118]:
url = 'http://apis.data.go.kr/1741000/EmergencyAssemblyArea_Earthquake2/getArea1List'
params ={'serviceKey' : decoding, 'pageNo' : pagenum, 'numOfRows' : numofrows, 'type' : 'json' }
response = requests.get(url, params=params)
jsondata = response.json()

In [119]:
jsondata['EarthquakeOutdoorsShelter'][0]

{'head': [{'totalCount': 11262},
  {'numOfRows': '1000', 'pageNo': '3', 'type': 'JSON'},
  {'RESULT': {'resultCode': 'INFO-0', 'resultMsg': 'NOMAL SERVICE'}}]}

In [129]:
jsondata['EarthquakeOutdoorsShelter'][1]
# output : {'row : [{옥외대피소 1 정보}, {옥외 대피소 2 정보}, ....]'}

{'row': [{'arcd': '4514000000',
   'acmdfclty_sn': 32,
   'ctprvn_nm': '전라북도',
   'sgg_nm': '익산시',
   'vt_acmdfclty_nm': '전북맹아학교 운동장',
   'rdnmadr_cd': '4514013600103580034060832',
   'bdong_cd': '4514013600',
   'hdong_cd': '4514067000',
   'dtl_adres': '전라북도 익산시 석암동   358-8',
   'fclty_ar': 1215,
   'xcord': '127.00581870538714',
   'ycord': '35.94204903741861'},
  {'arcd': '4514000000',
   'acmdfclty_sn': 106,
   'ctprvn_nm': '전라북도',
   'sgg_nm': '익산시',
   'vt_acmdfclty_nm': '익산부천중학교 운동장',
   'rdnmadr_cd': '',
   'bdong_cd': '4514012300',
   'hdong_cd': '4514012300',
   'dtl_adres': '전라북도 익산시 어양동   7-1',
   'fclty_ar': 1746,
   'xcord': '126.99038469617159',
   'ycord': '35.95295809936892'},
  {'arcd': '4514000000',
   'acmdfclty_sn': 104,
   'ctprvn_nm': '전라북도',
   'sgg_nm': '익산시',
   'vt_acmdfclty_nm': '이리영등중학교 운동장',
   'rdnmadr_cd': '',
   'bdong_cd': '4514012300',
   'hdong_cd': '4514012300',
   'dtl_adres': '전라북도 익산시 어양동   622-1',
   'fclty_ar': 2901,
   'xcord': '126.981513945

In [122]:
# jsondata['EarthquakeOutdoorsShelter'][1]['row']
# ouput : [{옥외대피소 1 정보}, {옥외 대피소 2 정보}, ....]

In [228]:

total_count = jsonData['EarthquakeOutdoorsShelter'][0]['head'][0]['totalCount']

NameError: name 'jsonData' is not defined